In [2]:
import os
import h5py
import numpy as np
import tensorflow as tf
import SnowDepth.architecture as ARCH
import SnowDepth.data_splitter as DS
import SnowDepth.evaluation as EVAL
from SnowDepth.config import HOLDOUT_AOI

In [3]:
# Path to the best models artifacts, which was saved during training

h5_path = "data/h5_dir/HSIC/data_HSIC.h5"
holdout_aoi = HOLDOUT_AOI
FINAL_NORM = "models/UNet_weights_and_norm/best_norm_stats.npz"
FINAL_WEIGHTS = "models/UNet_weights_and_norm/unet_best.weights.h5"

In [4]:
# Same as training
PATCH_SIZE = 128
LR = 1e-3
STRIDE = PATCH_SIZE

with np.load(FINAL_NORM) as f:
    mu, sigma = f["mean"], f["std"]

with h5py.File(h5_path, "r") as hf:
    C = hf[holdout_aoi]["features"].shape[-1]

model = ARCH.unet(input_shape=(PATCH_SIZE, PATCH_SIZE, C), base_filters=32)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
model.load_weights(FINAL_WEIGHTS)
print("Model + weights loaded.")

Model + weights loaded.


c:\Users\mathi\miniconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 130 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
# Get the holdout AOI data, with all tiles (no min_valid_frac)

(_, _, _), (_, _, _), (X_hold, y_hold, m_hold) = DS.DL_split(
    h5_path=h5_path,
    holdout_aoi=holdout_aoi,
    val_fraction=0.10,
    patch_size=PATCH_SIZE,
    stride=PATCH_SIZE,      
    min_valid_frac=0
)

In [6]:
scores = EVAL.evaluate_tiles(
    X_hold, y_hold, m_hold,
    model=model, mu=mu, sigma=sigma,
    batch_size=4,
)
print("Holdout (tiles only):", scores)

Holdout (tiles only): {'MAE': 0.6526486827698955, 'RMSE': 0.8039480577502189, 'R2': -0.8842022791075037, 'count': 237438.0}
